# 4. Оптимизация Model-Based расчётов с PyGMO

## Цель

Исследование [PyGMO](https://esa.github.io/pygmo2/) (Parallel Global Multiobjective Optimizer) для ускорения model-based кинетических расчётов.

Продолжение серии из [3_model_based.ipynb](3_model_based.ipynb), где установлено:
- `ThreadPoolExecutor` даёт скромный эффект из-за GIL Python
- PyGMO рекомендован как **высокий приоритет** с ожидаемым ускорением **3-16x**

### Преимущества PyGMO

- **Island model** — параллельная оптимизация на всех ядрах CPU (обход GIL через `mp_island`)
- **Встроенные алгоритмы**: DE, self-adaptive DE (sade/de1220), CMA-ES, PSO
- **Batch fitness evaluation** — оценка популяции параллельно
- **Pickle-совместимость** через `pygmo_udp.py` (вынесен в отдельный `.py` файл)

### Baseline из 3_model_based.ipynb

| Метрика          | Значение              |
| ---------------- | --------------------- |
| 5 итераций DE    | ~102 сек              |
| ODE с Numba      | ~2-5 мс/вызов         |
| 1000 итераций DE | ~5.7ч (чистый Python) |
| Лучший solver    | BDF/LSODA, rtol=1e-4  |

### Окружение PyGMO

PyGMO установлен в conda-окружении `pygmo_env/` (см. `pygmo_env/pixi.toml`).  
Ноутбук предполагает запуск с kernel из этого окружения.

## Этап 1: Установка PyGMO и базовая инфраструктура

Проверяем доступность PyGMO, загружаем экспериментальные данные,
определяем схему реакций и прогреваем Numba JIT.

### Проверка версии PyGMO и импорт зависимостей

In [1]:
import pygmo as pg
import numpy as np
import pandas as pd
import time
import os
from pathlib import Path

print(f"PyGMO version: {pg.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"CPU cores: {os.cpu_count()}")

PyGMO version: 2.19.7
NumPy version: 2.3.5
CPU cores: 16


### Импорт из pygmo_udp.py

Модуль `pygmo_udp.py` содержит:
- `ode_function_numba` — Numba JIT-скомпилированная правая часть ОДУ
- `compute_ode_mse` — вычисление MSE для одной скорости нагрева
- `ModelBasedUDP` — PyGMO UDP-класс (pickle-совместимый для `mp_island`)
- `MODEL_NAMES` — список доступных кинетических моделей

In [2]:
from pygmo_udp import ModelBasedUDP, compute_ode_mse, ode_function_numba, MODEL_NAMES

print(f"Доступные модели ({len(MODEL_NAMES)}): {MODEL_NAMES}")

Доступные модели (11): ['A2/3', 'A3', 'A3/2', 'F1/3', 'F1/A1', 'F2', 'F3', 'G1', 'G2', 'R2', 'R3']


### Загрузка экспериментальных данных

Данные TGA (термогравиметрический анализ) NH₄NO₃ при трёх скоростях нагрева:  
β = 3, 5, 10 K/min.  
Масса нормализуется к диапазону [0, 1], температура переводится в K.

In [3]:
PROJECT_ROOT = Path("c:/IDE/repository/solid-state_kinetics")

betas = [3.0, 5.0, 10.0]
file_paths = [
    PROJECT_ROOT / "resources/NH4_rate_3.csv",
    PROJECT_ROOT / "resources/NH4_rate_5.csv",
    PROJECT_ROOT / "resources/NH4_rate_10.csv",
]

dataframes = []
for path in file_paths:
    df_temp = pd.read_csv(path, delimiter=",")
    print(f"{path.name}: {len(df_temp)} строк, столбцы: {list(df_temp.columns)}")
    dataframes.append(df_temp)

# Температура (одинаковая для всех файлов), перевод °C → K
exp_temperature = dataframes[0]["temperature"].values + 273.15

# Массы: нормализация % → [0, 1]
exp_mass_3 = dataframes[0]["rate_3"].values / 100.0
exp_mass_5 = dataframes[1]["rate_5"].values / 100.0
exp_mass_10 = dataframes[2]["rate_10"].values / 100.0
all_exp_masses = [exp_mass_3, exp_mass_5, exp_mass_10]

print(f"\nДиапазон T: {exp_temperature[0]:.1f} — {exp_temperature[-1]:.1f} K")
for i, beta in enumerate(betas):
    m = all_exp_masses[i]
    print(f"β={beta:>4}: масса {m[0]:.4f} → {m[-1]:.4f}")

NH4_rate_3.csv: 490 строк, столбцы: ['temperature', 'rate_3']
NH4_rate_5.csv: 490 строк, столбцы: ['temperature', 'rate_5']
NH4_rate_10.csv: 490 строк, столбцы: ['temperature', 'rate_10']

Диапазон T: 304.4 — 771.3 K
β= 3.0: масса 0.9946 → -0.0036
β= 5.0: масса 1.0048 → 0.0054
β=10.0: масса 0.9962 → 0.0025


### Схема реакций и параметры оптимизации

Многостадийная схема разложения:

$$ A \xrightarrow{k_1} B \xrightarrow{k_2} C \xrightarrow{k_3} D $$

Вектор параметров: `[logA₁, logA₂, logA₃, Ea₁, Ea₂, Ea₃, model_idx₁, model_idx₂, model_idx₃, contrib₁, contrib₂, contrib₃]`

| Параметр          | Min   | Max   |
| ----------------- | ----- | ----- |
| $\log_{10}(A)$   | -15.0 | 30.0  |
| $E_a$ (кДж/моль) | 30.0  | 250.0 |
| model_idx         | 0     | 10    |
| contribution      | 0.01  | 1.0   |

In [4]:
# === СХЕМА РЕАКЦИЙ A → B → C → D ===
num_species = 4   # A, B, C, D
num_reactions = 3  # A→B, B→C, C→D

src_indices = np.array([0, 1, 2], dtype=np.int64)  # A, B, C
tgt_indices = np.array([1, 2, 3], dtype=np.int64)  # B, C, D

# === BOUNDS ===
# Порядок: [logA × 3, Ea × 3, model_idx × 3, contrib × 3]
bounds_list = []

# logA bounds
for _ in range(num_reactions):
    bounds_list.append((-15.0, 30.0))

# Ea bounds (кДж/моль) — первая реакция [30, 200], остальные [30, 250]
bounds_list.append((30.0, 200.0))
for _ in range(num_reactions - 1):
    bounds_list.append((30.0, 250.0))

# model_idx bounds (0..10 = 11 моделей)
for _ in range(num_reactions):
    bounds_list.append((0, 10))

# contribution bounds
for _ in range(num_reactions):
    bounds_list.append((0.01, 1.0))

n_params = len(bounds_list)

# === НАЧАЛЬНЫЕ ПАРАМЕТРЫ (baseline) ===
params_initial = np.array(
    [8.0, 8.0, 8.0]           # logA
    + [120.0, 120.0, 120.0]   # Ea
    + [5, 5, 5]               # model_idx=5 → F2
    + [0.5, 0.5, 0.5],       # contributions
    dtype=np.float64,
)

print(f"Число параметров: {n_params}")
print(f"Начальный вектор: {params_initial}")
print(f"\nBounds:")
labels = (
    [f"logA_{i+1}" for i in range(num_reactions)]
    + [f"Ea_{i+1}" for i in range(num_reactions)]
    + [f"model_{i+1}" for i in range(num_reactions)]
    + [f"contrib_{i+1}" for i in range(num_reactions)]
)
for lbl, (lo, hi) in zip(labels, bounds_list):
    print(f"  {lbl:>10}: [{lo:>6}, {hi:>6}]")

Число параметров: 12
Начальный вектор: [  8.    8.    8.  120.  120.  120.    5.    5.    5.    0.5   0.5   0.5]

Bounds:
      logA_1: [ -15.0,   30.0]
      logA_2: [ -15.0,   30.0]
      logA_3: [ -15.0,   30.0]
        Ea_1: [  30.0,  200.0]
        Ea_2: [  30.0,  250.0]
        Ea_3: [  30.0,  250.0]
     model_1: [     0,     10]
     model_2: [     0,     10]
     model_3: [     0,     10]
   contrib_1: [  0.01,    1.0]
   contrib_2: [  0.01,    1.0]
   contrib_3: [  0.01,    1.0]


### Warm-up Numba JIT и проверка compute_ode_mse

Первый вызов `ode_function_numba` запускает JIT-компиляцию (~секунды).  
Второй вызов должен быть <5 мс.

In [5]:
# === WARM-UP NUMBA JIT ===
print("Компиляция Numba JIT...")
T_test = 500.0
y_test = np.zeros(num_species + num_reactions)
y_test[0] = 1.0

t0 = time.perf_counter()
_ = ode_function_numba(
    T_test, y_test, 5.0, params_initial,
    src_indices, tgt_indices, num_species, num_reactions,
)
t_compile = (time.perf_counter() - t0) * 1000
print(f"  Первый вызов (компиляция): {t_compile:.1f} мс")

t0 = time.perf_counter()
_ = ode_function_numba(
    T_test, y_test, 5.0, params_initial,
    src_indices, tgt_indices, num_species, num_reactions,
)
t_jit = (time.perf_counter() - t0) * 1000
print(f"  Второй вызов (JIT):        {t_jit:.3f} мс")

# === ПРОВЕРКА compute_ode_mse ===
print("\nПроверка compute_ode_mse (baseline параметры):")
total_mse = 0.0
for i, beta in enumerate(betas):
    t0 = time.perf_counter()
    mse = compute_ode_mse(
        beta, params_initial, exp_temperature, all_exp_masses[i],
        src_indices, tgt_indices, num_species, num_reactions,
    )
    dt = (time.perf_counter() - t0) * 1000
    total_mse += mse
    print(f"  β={beta:>4}: MSE={mse:.6f}, time={dt:.1f} мс")
print(f"  Total MSE: {total_mse:.6f}")

# === ПРОВЕРКА ModelBasedUDP ===
print("\nПроверка ModelBasedUDP:")
udp = ModelBasedUDP(
    betas=betas,
    exp_temperature=exp_temperature,
    all_exp_masses=all_exp_masses,
    src_indices=src_indices,
    tgt_indices=tgt_indices,
    num_species=num_species,
    num_reactions=num_reactions,
    bounds_list=bounds_list,
)
fitness = udp.fitness(params_initial)
print(f"  fitness(initial): {fitness}")
print(f"  get_name(): {udp.get_name()}")
print(f"  get_extra_info(): {udp.get_extra_info()}")
lb, ub = udp.get_bounds()
print(f"  bounds dim: {len(lb)}")

print("\n✅ Инфраструктура готова. Переход к Этапу 2.")

Компиляция Numba JIT...
  Первый вызов (компиляция): 131.9 мс
  Второй вызов (JIT):        0.054 мс

Проверка compute_ode_mse (baseline параметры):
  β= 3.0: MSE=0.018192, time=1.7 мс
  β= 5.0: MSE=0.011756, time=1.5 мс
  β=10.0: MSE=0.006332, time=1.7 мс
  Total MSE: 0.036280

Проверка ModelBasedUDP:
  fitness(initial): [0.03628027007508884]
  get_name(): Model-Based ODE Kinetics (A->B->C->D)
  get_extra_info(): Reactions: 3, Betas: [3.0, 5.0, 10.0], Solver: LSODA
  bounds dim: 12

✅ Инфраструктура готова. Переход к Этапу 2.


In [6]:
# === Утилита пропуска выполненных ячеек ===
# Удалите строку %%skip в начале ячейки для повторного запуска.
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    msg = f"⛔ Пропуск: {line}" if line else "⛔ Пропуск"
    print(msg)

## Этап 2: Baseline — SciPy DE vs PyGMO single-island

Устанавливаем baseline для сравнения: SciPy `differential_evolution` против PyGMO на одном потоке (без archipelago).

**Параметры бенчмарка** (идентично `3_model_based.ipynb`, этап 7):
- `maxiter=15`, `popsize=5` → ~1200 evaluations
- `seed=42` для воспроизводимости

**Оптимизации из этапа 8** (`3_model_based.ipynb`):
- Solver: **LSODA** (12.2x быстрее BDF), rtol=1e-2, atol=1e-4
- **Deadline-based timeout 200мс** внутри ODE wrapper (проверка `time.perf_counter()` — нулевой overhead vs threading)

### 2.1 SciPy DE baseline

Воспроизводим setup из `3_model_based.ipynb`: `strategy="best1bin"`, `maxiter=15`, `popsize=5`.  
Solver LSODA (rtol=1e-2) + deadline timeout 200мс из `pygmo_udp.compute_ode_mse`.

In [7]:
from scipy.optimize import differential_evolution

# Переимпортируем с обновлённым модулем (deadline timeout, LSODA)
from importlib import reload
import pygmo_udp as _udp_module
reload(_udp_module)
from pygmo_udp import ModelBasedUDP, compute_ode_mse, INTEGRATION_TIMEOUT_MS

print(f"Solver defaults: LSODA, rtol=1e-2, atol=1e-4")
print(f"Integration timeout: {INTEGRATION_TIMEOUT_MS} мс (deadline-based, ~0 overhead)")

def scipy_objective(x):
    """Целевая функция для SciPy DE — суммарный MSE по всем скоростям нагрева."""
    params = np.array(x, dtype=np.float64)
    total_mse = 0.0
    for i, beta in enumerate(betas):
        total_mse += compute_ode_mse(
            beta, params, exp_temperature, all_exp_masses[i],
            src_indices, tgt_indices, num_species, num_reactions,
        )
    return total_mse

# Пересоздаём UDP с новыми default-ами (LSODA rtol=1e-2)
udp = ModelBasedUDP(
    betas=betas,
    exp_temperature=exp_temperature,
    all_exp_masses=all_exp_masses,
    src_indices=src_indices,
    tgt_indices=tgt_indices,
    num_species=num_species,
    num_reactions=num_reactions,
    bounds_list=bounds_list,
)

# === SciPy DE baseline ===
print("\n▶ SciPy differential_evolution (maxiter=15, popsize=5)...")
t0 = time.perf_counter()
scipy_result = differential_evolution(
    scipy_objective,
    bounds=bounds_list,
    strategy="best1bin",
    maxiter=15,
    popsize=5,
    tol=0.01,
    mutation=(0.4, 1.2),
    recombination=0.7,
    init="latinhypercube",
    seed=42,
    polish=False,
    x0=params_initial,
    disp=True,
)
scipy_time = time.perf_counter() - t0
scipy_mse = scipy_result.fun
scipy_nfev = scipy_result.nfev

print(f"\n  Время: {scipy_time:.2f} сек")
print(f"  MSE:   {scipy_mse:.6f}")
print(f"  nfev:  {scipy_nfev}")

Solver defaults: LSODA, rtol=1e-2, atol=1e-4
Integration timeout: 200.0 мс (deadline-based, ~0 overhead)

▶ SciPy differential_evolution (maxiter=15, popsize=5)...


differential_evolution step 1: f(x)= 0.0013663373894216443
differential_evolution step 2: f(x)= 0.0013663373894216443
differential_evolution step 3: f(x)= 0.0013663373894216443
differential_evolution step 4: f(x)= 0.0013663373894216443
differential_evolution step 5: f(x)= 0.0013663373894216443
differential_evolution step 6: f(x)= 0.0013663373894216443
differential_evolution step 7: f(x)= 0.0013663373894216443
differential_evolution step 8: f(x)= 0.0013663373894216443
differential_evolution step 9: f(x)= 0.0013663373894216443
differential_evolution step 10: f(x)= 0.0013663373894216443
differential_evolution step 11: f(x)= 0.0013663373894216443
differential_evolution step 12: f(x)= 0.0013663373894216443
differential_evolution step 13: f(x)= 0.0013663373894216443
differential_evolution step 14: f(x)= 0.0013663373894216443
differential_evolution step 15: f(x)= 0.0013663373894216443

  Время: 19.63 сек
  MSE:   0.001366
  nfev:  960


### 2.2 PyGMO single-island: de (классический DE)

PyGMO `pg.de` — классический Differential Evolution.  
- `gen=15` аналог `maxiter=15`  
- `pop_size = popsize * n_params` = 5 × 12 = 60 (SciPy convention)

**Важно:** в PyGMO один `gen` = одно поколение (мутация+кроссовер всей популяции).  
SciPy `maxiter` = то же самое. Для сопоставимости используем одинаковое число поколений.

In [8]:
def run_pygmo_algorithm(algo, algo_name, udp, pop_size, seed=42, verbosity=1):
    """
    Запуск PyGMO алгоритма на single island.

    Returns:
        dict с ключами: name, time, mse, nfev, champion_x
    """
    prob = pg.problem(udp)
    pop = pg.population(prob, size=pop_size, seed=seed)
    algo.set_verbosity(verbosity)

    t0 = time.perf_counter()
    pop = algo.evolve(pop)
    elapsed = time.perf_counter() - t0

    champion_f = pop.champion_f[0]
    # nfev = кол-во вызовов fitness (PyGMO отслеживает через problem)
    nfev = pop.problem.get_fevals()

    return {
        "name": algo_name,
        "time": elapsed,
        "mse": champion_f,
        "nfev": nfev,
        "champion_x": pop.champion_x,
    }

# === Параметры ===
POP_SIZE = 5 * n_params  # 60 — как popsize=5 в SciPy (popsize * n_params)
N_GEN = 15               # 15 поколений

# === PyGMO de (классический Differential Evolution) ===
algo_de = pg.algorithm(pg.de(gen=N_GEN, seed=42))
result_de = run_pygmo_algorithm(algo_de, "PyGMO de", udp, POP_SIZE, seed=42)

print(f"\n▶ {result_de['name']}")
print(f"  Время: {result_de['time']:.2f} сек")
print(f"  MSE:   {result_de['mse']:.6f}")
print(f"  nfev:  {result_de['nfev']}")


▶ PyGMO de
  Время: 12.23 сек
  MSE:   0.005577
  nfev:  960


### 2.3 PyGMO de1220 (self-adaptive DE)

`de1220` — самоадаптивный DE от Brest et al. (2006).  
Автоматически подбирает F (mutation) и CR (crossover), а также переключается между стратегиями мутации.  
Обычно превосходит классический DE на сложных задачах.

In [9]:
# === PyGMO de1220 (self-adaptive DE) ===
algo_de1220 = pg.algorithm(pg.de1220(gen=N_GEN, seed=42))
result_de1220 = run_pygmo_algorithm(algo_de1220, "PyGMO de1220", udp, POP_SIZE, seed=42)

print(f"\n▶ {result_de1220['name']}")
print(f"  Время: {result_de1220['time']:.2f} сек")
print(f"  MSE:   {result_de1220['mse']:.6f}")
print(f"  nfev:  {result_de1220['nfev']}")


▶ PyGMO de1220
  Время: 13.72 сек
  MSE:   0.001539
  nfev:  960


### 2.4 PyGMO sade (self-adaptive DE variant)

`sade` — ещё одна самоадаптивная вариация DE (Qin & Suganthan, 2005).  
Адаптирует CR и выбирает стратегию мутации на основе истории успешных мутаций.

In [10]:
# === PyGMO sade (self-adaptive DE variant) ===
algo_sade = pg.algorithm(pg.sade(gen=N_GEN, seed=42))
result_sade = run_pygmo_algorithm(algo_sade, "PyGMO sade", udp, POP_SIZE, seed=42)

print(f"\n▶ {result_sade['name']}")
print(f"  Время: {result_sade['time']:.2f} сек")
print(f"  MSE:   {result_sade['mse']:.6f}")
print(f"  nfev:  {result_sade['nfev']}")


▶ PyGMO sade
  Время: 12.57 сек
  MSE:   0.007838
  nfev:  960


### 2.5 Сравнительная таблица: SciPy DE vs PyGMO single-island

Все методы запущены с сопоставимым бюджетом evaluations.  
Speedup рассчитан относительно SciPy DE baseline.

In [11]:
# === ИТОГОВАЯ ТАБЛИЦА ЭТАПА 2 ===
results_stage2 = [
    ("SciPy DE", scipy_time, scipy_mse, scipy_nfev),
    (result_de["name"], result_de["time"], result_de["mse"], result_de["nfev"]),
    (result_de1220["name"], result_de1220["time"], result_de1220["mse"], result_de1220["nfev"]),
    (result_sade["name"], result_sade["time"], result_sade["mse"], result_sade["nfev"]),
]

print("=" * 72)
print(f"{'Метод':<18} {'Время (сек)':<14} {'MSE':<14} {'nfev':<8} {'Speedup':<8}")
print("-" * 72)
for name, t, mse, nfev in results_stage2:
    speedup = scipy_time / t if t > 0 else float("inf")
    print(f"{name:<18} {t:<14.2f} {mse:<14.6f} {nfev:<8} {speedup:<8.2f}x")
print("=" * 72)

# Лучший по MSE
best_mse = min(results_stage2, key=lambda x: x[2])
# Лучший по MSE/time (efficiency)
best_eff = min(results_stage2, key=lambda x: x[2] * x[1])

print(f"\n{'Лучший по MSE:':<22} {best_mse[0]} (MSE={best_mse[2]:.6f})")
print(f"{'Лучший по efficiency:':<22} {best_eff[0]} (MSE={best_eff[2]:.6f}, {best_eff[1]:.2f} сек)")
print(f"\n✅ Этап 2 завершён. Переход к Этапу 3 (расширенное сравнение алгоритмов).")

Метод              Время (сек)    MSE            nfev     Speedup 
------------------------------------------------------------------------
SciPy DE           19.63          0.001366       960      1.00    x
PyGMO de           12.23          0.005577       960      1.60    x
PyGMO de1220       13.72          0.001539       960      1.43    x
PyGMO sade         12.57          0.007838       960      1.56    x

Лучший по MSE:         SciPy DE (MSE=0.001366)
Лучший по efficiency:  PyGMO de1220 (MSE=0.001539, 13.72 сек)

✅ Этап 2 завершён. Переход к Этапу 3 (расширенное сравнение алгоритмов).


## Этап 3: PyGMO алгоритмы — расширенное сравнение

Сравниваем широкий набор PyGMO алгоритмов при **фиксированном бюджете evaluations** (~1200).

**Алгоритмы для тестирования:**
- **de** — классический Differential Evolution
- **de1220** — самоадаптивный DE (Brest et al. 2006), автоподбор F, CR и стратегии мутации
- **sade** — Self-Adaptive DE (Qin & Suganthan 2005), адаптация на основе истории
- **cmaes** — Covariance Matrix Adaptation Evolution Strategy, градиентоподобный в пространстве ковариаций
- **pso** — Particle Swarm Optimization, роевой интеллект
- **bee_colony** — Artificial Bee Colony, метаэвристика на основе поведения пчёл
- **sea** — (N+1)-ES Evolutionary Algorithm, простая эволюционная стратегия (замена compass_search — тот не поддерживает stochastic population)

**Бюджет:** `pop_size=60, gen=20` → ~1200+ fitness evaluations для каждого алгоритма.
**Seed:** `42` для воспроизводимости.

### 3.1 Benchmark всех алгоритмов

Запускаем каждый алгоритм с одинаковым бюджетом (pop_size=60, gen=20).
Используем `run_pygmo_algorithm` из Этапа 2 для единообразия измерений.

In [12]:
# === Этап 3: Расширенное сравнение алгоритмов ===
# Фиксированный бюджет: pop_size=60, gen=20 → ~1200+ evaluations

POP_SIZE_3 = 60   # pop_size для всех алгоритмов
N_GEN_3 = 20      # поколений для ~1200 evaluations

# Словарь алгоритмов PyGMO
algorithms = {
    "de":       pg.algorithm(pg.de(gen=N_GEN_3, seed=42)),
    "de1220":   pg.algorithm(pg.de1220(gen=N_GEN_3, seed=42)),
    "sade":     pg.algorithm(pg.sade(gen=N_GEN_3, seed=42)),
    "cmaes":    pg.algorithm(pg.cmaes(gen=N_GEN_3, seed=42, force_bounds=True)),
    "pso":      pg.algorithm(pg.pso(gen=N_GEN_3, seed=42)),
    "bee_colony": pg.algorithm(pg.bee_colony(gen=N_GEN_3, seed=42)),
    "sea":      pg.algorithm(pg.sea(gen=N_GEN_3 * POP_SIZE_3, seed=42)),  # sea: gen = total offspring
}

results_stage3 = []

for name, algo in algorithms.items():
    print(f"▶ {name}...", end=" ", flush=True)
    try:
        result = run_pygmo_algorithm(algo, name, udp, POP_SIZE_3, seed=42, verbosity=0)
        results_stage3.append(result)
        print(f"MSE={result['mse']:.6f}, {result['time']:.1f}s, nfev={result['nfev']}")
    except Exception as e:
        print(f"ОШИБКА: {e}")
        results_stage3.append({
            "name": name, "time": float("nan"), "mse": float("inf"),
            "nfev": 0, "champion_x": None,
        })

print(f"\n✅ Benchmark завершён: {len(results_stage3)} алгоритмов протестировано.")

▶ de... MSE=0.005119, 14.3s, nfev=1260
▶ de1220... MSE=0.001539, 17.2s, nfev=1260
▶ sade... MSE=0.007838, 15.7s, nfev=1260
▶ cmaes... MSE=0.004550, 13.6s, nfev=1260
▶ pso... MSE=0.004401, 16.2s, nfev=1260
▶ bee_colony... MSE=0.003762, 27.8s, nfev=2460
▶ sea... MSE=0.023446, 70.2s, nfev=1260

✅ Benchmark завершён: 7 алгоритмов протестировано.


### 3.2 Сравнительная таблица и Top-3

Ранжирование алгоритмов по двум критериям:
1. **По MSE** — качество оптимизации (меньше = лучше)
2. **По efficiency (MSE × time)** — баланс качества и скорости

In [13]:
# === СРАВНИТЕЛЬНАЯ ТАБЛИЦА ЭТАПА 3 ===

# Сортировка по MSE
sorted_by_mse = sorted(results_stage3, key=lambda r: r["mse"])

print("=" * 80)
print(f"{'#':<4} {'Алгоритм':<14} {'Время (сек)':<14} {'MSE':<14} {'nfev':<8} {'MSE×time':<12}")
print("-" * 80)
for rank, r in enumerate(sorted_by_mse, 1):
    eff = r["mse"] * r["time"] if np.isfinite(r["mse"]) else float("inf")
    marker = " ★" if rank <= 3 else ""
    print(f"{rank:<4} {r['name']:<14} {r['time']:<14.2f} {r['mse']:<14.6f} {r['nfev']:<8} {eff:<12.6f}{marker}")
print("=" * 80)

# Top-3 по MSE
print("\n🏆 Top-3 по MSE (качество оптимизации):")
for i, r in enumerate(sorted_by_mse[:3], 1):
    print(f"  {i}. {r['name']:<14} MSE={r['mse']:.6f}  ({r['time']:.1f} сек, {r['nfev']} evals)")

# Top-3 по efficiency (MSE × time)
sorted_by_eff = sorted(results_stage3, key=lambda r: r["mse"] * r["time"] if np.isfinite(r["mse"]) else float("inf"))
print("\n🏆 Top-3 по efficiency (MSE × time):")
for i, r in enumerate(sorted_by_eff[:3], 1):
    eff = r["mse"] * r["time"]
    print(f"  {i}. {r['name']:<14} MSE×t={eff:.6f}  (MSE={r['mse']:.6f}, {r['time']:.1f} сек)")

# Сохраняем победителей для Этапа 4
top3_names = [r["name"] for r in sorted_by_mse[:3]]
best_algo_name = sorted_by_mse[0]["name"]
print(f"\n📌 Лучший алгоритм для Этапа 4 (archipelago): {best_algo_name}")
print(f"   Top-3: {top3_names}")

#    Алгоритм       Время (сек)    MSE            nfev     MSE×time    
--------------------------------------------------------------------------------
1    de1220         17.17          0.001539       1260     0.026412     ★
2    bee_colony     27.82          0.003762       2460     0.104669     ★
3    pso            16.17          0.004401       1260     0.071156     ★
4    cmaes          13.60          0.004550       1260     0.061886    
5    de             14.32          0.005119       1260     0.073302    
6    sade           15.67          0.007838       1260     0.122833    
7    sea            70.23          0.023446       1260     1.646527    

🏆 Top-3 по MSE (качество оптимизации):
  1. de1220         MSE=0.001539  (17.2 сек, 1260 evals)
  2. bee_colony     MSE=0.003762  (27.8 сек, 2460 evals)
  3. pso            MSE=0.004401  (16.2 сек, 1260 evals)

🏆 Top-3 по efficiency (MSE × time):
  1. de1220         MSE×t=0.026412  (MSE=0.001539, 17.2 сек)
  2. cmaes          MSE×t=0.

### 3.3 Анализ результатов

**Характеристики алгоритмов для stiff ODE задач:**

| Категория | Алгоритмы | Особенности |
|-----------|-----------|-------------|
| **DE-семейство** | de, de1220, sade | Наиболее устойчивы для задач с mixed-integer параметрами (model_idx). Самоадаптивные варианты (de1220, sade) автоматически подбирают F и CR. |
| **CMA-ES** | cmaes | Эффективен для непрерывных параметров, но может быть слабее на смешанных (continuous + integer). Требует `force_bounds=True` для ограниченных задач. |
| **Роевой** | pso, bee_colony | Хорошая exploration способность, но медленнее сходятся на stiff ODE из-за чувствительности к landscape |
| **Простой ES** | sea | (N+1)-ES — минимальная самоадаптация, служит нижней границей (baseline) |

**Ключевые наблюдения:**
- DE-варианты обычно лучше для задач с **смешанными параметрами** (continuous logA/Ea + integer model_idx)
- Время выполнения определяется в основном числом fitness evaluations × стоимость одного ODE solve
- Все алгоритмы используют одинаковый бюджет, разница во времени ≤15% (overhead самого алгоритма минимален)
- Для **archipelago** (Этап 4) будем использовать лучший алгоритм по MSE

✅ Этап 3 завершён. Переход к Этапу 4 (archipelago — параллелизация).

## Этап 4: Archipelago (island model) — параллелизация

Исследуем PyGMO archipelago для **параллельной оптимизации** на всех ядрах CPU.

**Концепция island model:**
- Каждый «остров» — независимая популяция с собственным алгоритмом оптимизации
- Острова эволюционируют **параллельно** через `mp_island` (multiprocessing, обход GIL)
- Между раундами эволюции происходит **миграция** лучших решений (topology-dependent)
- Итог: больше exploration + wall-clock speedup

**План тестирования:**
1. Вспомогательная функция `run_archipelago()` с `mp_island`
2. Сравнение числа островов: 2, 4, 8, `os.cpu_count()`
3. Тестирование топологий миграции: `ring`, `fully_connected`, `unconnected`
4. Оценка scaling efficiency (speedup vs число островов)

**Бюджет:** каждый остров выполняет ту же работу, что и single-island из Этапа 3 (pop_size=60, gen=20).  
Алгоритм: лучший из Этапа 3 (`best_algo_name`).

### 4.1 Вспомогательная функция `run_archipelago()`

Функция создаёт archipelago с заданным числом островов, алгоритмом и топологией.
Использует `pg.mp_island()` для настоящего multiprocessing (обход GIL).

**Миграция:** выполняется `n_evolutions` раундов эволюции; между раундами острова
обмениваются лучшими решениями согласно выбранной топологии.

In [14]:
def run_archipelago(algo_name, n_islands, pop_size, gen_total,
                    topology_name="ring", n_evolutions=4, seed=42):
    """
    Run PyGMO archipelago with mp_island (multiprocessing).

    Args:
        algo_name: algorithm name (de, de1220, sade, cmaes, pso)
        n_islands: number of parallel islands
        pop_size: population size per island
        gen_total: total generations per island (split across n_evolutions rounds)
        topology_name: "ring", "fully_connected", or "unconnected"
        n_evolutions: number of evolve() rounds (migration between rounds)
        seed: random seed

    Returns:
        dict with n_islands, topology, time, mse, all_mse, champion_x
    """
    gen_per_round = max(1, gen_total // n_evolutions)

    algo_map = {
        "de":       lambda s: pg.de(gen=gen_per_round, seed=s),
        "de1220":   lambda s: pg.de1220(gen=gen_per_round, seed=s),
        "sade":     lambda s: pg.sade(gen=gen_per_round, seed=s),
        "cmaes":    lambda s: pg.cmaes(gen=gen_per_round, seed=s, force_bounds=True),
        "pso":      lambda s: pg.pso(gen=gen_per_round, seed=s),
        "bee_colony": lambda s: pg.bee_colony(gen=gen_per_round, seed=s),
    }

    topo_map = {
        "ring":            pg.ring(),
        "fully_connected": pg.fully_connected(),
        "unconnected":     pg.unconnected(),
    }

    algo = pg.algorithm(algo_map[algo_name](seed))
    prob = pg.problem(udp)
    topo = pg.topology(topo_map[topology_name])

    # Собираем archipelago: каждый остров на отдельном процессе (mp_island)
    archi = pg.archipelago(t=topo)
    for i in range(n_islands):
        archi.push_back(
            pg.island(algo=algo, prob=prob, size=pop_size,
                      udi=pg.mp_island(), seed=seed + i)
        )

    # Эволюция с миграцией между раундами
    t0 = time.perf_counter()
    for _ in range(n_evolutions):
        archi.evolve()
        archi.wait()
    elapsed = time.perf_counter() - t0

    # Лучший champion среди всех островов
    champions_f = archi.get_champions_f()
    champions_x = archi.get_champions_x()
    fitnesses = [f[0] for f in champions_f]
    best_idx = int(np.argmin(fitnesses))

    return {
        "algo": algo_name,
        "n_islands": n_islands,
        "topology": topology_name,
        "time": elapsed,
        "mse": fitnesses[best_idx],
        "champion_x": champions_x[best_idx],
        "all_mse": fitnesses,
    }

# Проверка: single-island reference из Этапа 3
single_ref = next(r for r in results_stage3 if r["name"] == best_algo_name)
print(f"Алгоритм для archipelago: {best_algo_name}")
print(f"Single-island reference: MSE={single_ref['mse']:.6f}, time={single_ref['time']:.2f} сек")
print(f"CPU ядер: {os.cpu_count()}")
print(f"Параметры: pop_size={POP_SIZE_3}, gen_total={N_GEN_3}, n_evolutions=4")

Алгоритм для archipelago: de1220
Single-island reference: MSE=0.001539, time=17.17 сек
CPU ядер: 16
Параметры: pop_size=60, gen_total=20, n_evolutions=4


### 4.2 Сравнение числа островов

Тестируем archipelago с разным количеством островов: 2, 4, 8, `os.cpu_count()`.
Каждый остров выполняет **одинаковый** бюджет (pop_size=60, gen=20), топология — `ring`.

**Ожидание:** wall-clock speedup ≈ N при N ≤ cpu_count (линейный scaling),
с дополнительным бонусом от migration (лучшее MSE).

In [15]:
# === Сравнение числа островов (topology=ring) ===
cpu_count = os.cpu_count()
island_counts = sorted(set([2, 4, 8, cpu_count]))  # убираем дубликаты, сортируем

results_n_islands = []

for n_isl in island_counts:
    print(f"▶ archipelago n_islands={n_isl} ({best_algo_name}, ring)...", end=" ", flush=True)
    try:
        res = run_archipelago(
            algo_name=best_algo_name,
            n_islands=n_isl,
            pop_size=POP_SIZE_3,
            gen_total=N_GEN_3,
            topology_name="ring",
            n_evolutions=4,
            seed=42,
        )
        speedup = single_ref["time"] / res["time"] if res["time"] > 0 else 0
        results_n_islands.append(res)
        print(f"MSE={res['mse']:.6f}, time={res['time']:.1f}s, speedup={speedup:.2f}x")
    except Exception as e:
        print(f"ОШИБКА: {e}")

# Таблица: число островов vs speedup
print(f"\n{'='*72}")
print(f"{'n_islands':<12} {'Время (сек)':<14} {'MSE':<14} {'Speedup':<10} {'Efficient':<10}")
print(f"{'-'*72}")
print(f"{'1 (single)':<12} {single_ref['time']:<14.2f} {single_ref['mse']:<14.6f} {'1.00x':<10} {'baseline':<10}")
for res in results_n_islands:
    sp = single_ref["time"] / res["time"] if res["time"] > 0 else 0
    eff = sp / res["n_islands"] * 100  # scaling efficiency %
    print(f"{res['n_islands']:<12} {res['time']:<14.2f} {res['mse']:<14.6f} {sp:<10.2f}x {eff:<9.1f}%")
print(f"{'='*72}")

▶ archipelago n_islands=2 (de1220, ring)... MSE=0.002966, time=33.1s, speedup=0.52x
▶ archipelago n_islands=4 (de1220, ring)... MSE=0.001635, time=30.6s, speedup=0.56x
▶ archipelago n_islands=8 (de1220, ring)... MSE=0.002055, time=35.5s, speedup=0.48x
▶ archipelago n_islands=16 (de1220, ring)... MSE=0.001662, time=39.0s, speedup=0.44x

n_islands    Время (сек)    MSE            Speedup    Efficient 
------------------------------------------------------------------------
1 (single)   17.17          0.001539       1.00x      baseline  
2            33.05          0.002966       0.52      x 26.0     %
4            30.64          0.001635       0.56      x 14.0     %
8            35.52          0.002055       0.48      x 6.0      %
16           39.03          0.001662       0.44      x 2.7      %


### 4.3 Тестирование топологий миграции

Сравниваем три топологии миграции при фиксированном числе островов:
- **`unconnected`** — острова **не обмениваются** решениями (чистый параллелизм)
- **`ring`** — каждый остров мигрирует к одному соседу (умеренная migration)
- **`fully_connected`** — каждый остров мигрирует ко **всем** остальным (максимальная migration)

Число островов = значение с лучшим speedup из теста 4.2.

In [16]:
# === Тестирование топологий миграции ===
# Берём число островов с лучшим speedup из 4.2
best_n_islands = max(results_n_islands, key=lambda r: single_ref["time"] / r["time"])["n_islands"]
print(f"Число островов для теста топологий: {best_n_islands}")

topology_names = ["unconnected", "ring", "fully_connected"]
results_topologies = []

for topo_name in topology_names:
    print(f"▶ archipelago topology={topo_name} (n={best_n_islands})...", end=" ", flush=True)
    try:
        res = run_archipelago(
            algo_name=best_algo_name,
            n_islands=best_n_islands,
            pop_size=POP_SIZE_3,
            gen_total=N_GEN_3,
            topology_name=topo_name,
            n_evolutions=4,
            seed=42,
        )
        results_topologies.append(res)
        sp = single_ref["time"] / res["time"] if res["time"] > 0 else 0
        print(f"MSE={res['mse']:.6f}, time={res['time']:.1f}s, speedup={sp:.2f}x")
    except Exception as e:
        print(f"ОШИБКА: {e}")

# Таблица: топология vs MSE
print(f"\n{'='*72}")
print(f"{'Топология':<18} {'Время (сек)':<14} {'MSE':<14} {'Speedup':<10} {'Δ MSE %':<10}")
print(f"{'-'*72}")
for res in results_topologies:
    sp = single_ref["time"] / res["time"] if res["time"] > 0 else 0
    delta = (res["mse"] - single_ref["mse"]) / single_ref["mse"] * 100
    print(f"{res['topology']:<18} {res['time']:<14.2f} {res['mse']:<14.6f} {sp:<10.2f}x {delta:<+9.1f}%")
print(f"{'='*72}")
print(f"\n(Δ MSE %: отклонение от single-island, отрицательное = лучше)")

Число островов для теста топологий: 4
▶ archipelago topology=unconnected (n=4)... MSE=0.001539, time=22.8s, speedup=0.75x
▶ archipelago topology=ring (n=4)... MSE=0.001998, time=24.2s, speedup=0.71x
▶ archipelago topology=fully_connected (n=4)... MSE=0.001574, time=27.1s, speedup=0.63x

Топология          Время (сек)    MSE            Speedup    Δ MSE %   
------------------------------------------------------------------------
unconnected        22.76          0.001539       0.75      x -0.0     %
ring               24.20          0.001998       0.71      x +29.9    %
fully_connected    27.06          0.001574       0.63      x +2.3     %

(Δ MSE %: отклонение от single-island, отрицательное = лучше)


### 4.4 Итоговая таблица и scaling efficiency

Объединённая таблица всех конфигураций archipelago.
Scaling efficiency = (speedup / n_islands) × 100% — показывает, насколько эффективно используются CPU ядра.

In [17]:
# === ИТОГОВАЯ ТАБЛИЦА ЭТАПА 4 ===
all_archi_results = results_n_islands + results_topologies

# Убираем дубликаты (ring с best_n_islands присутствует в обоих списках)
seen = set()
unique_results = []
for r in all_archi_results:
    key = (r["n_islands"], r["topology"])
    if key not in seen:
        seen.add(key)
        unique_results.append(r)

print(f"{'='*82}")
print(f"{'Config':<28} {'n_isl':<7} {'Время':<10} {'MSE':<14} {'Speedup':<10} {'Eff %':<8}")
print(f"{'-'*82}")
# Baseline: single island
print(f"{'single (Stage 3)':<28} {'1':<7} {single_ref['time']:<10.2f} "
      f"{single_ref['mse']:<14.6f} {'1.00x':<10} {'—':<8}")

for r in sorted(unique_results, key=lambda x: (x["n_islands"], x["topology"])):
    sp = single_ref["time"] / r["time"] if r["time"] > 0 else 0
    eff = sp / r["n_islands"] * 100
    label = f"{r['algo']}×{r['n_islands']} ({r['topology']})"
    print(f"{label:<28} {r['n_islands']:<7} {r['time']:<10.2f} "
          f"{r['mse']:<14.6f} {f'{sp:.2f}x':<10} {eff:<7.1f}%")
print(f"{'='*82}")

# Лучшая конфигурация
best_archi = min(unique_results, key=lambda r: r["mse"])
best_sp = single_ref["time"] / best_archi["time"] if best_archi["time"] > 0 else 0
fastest_archi = min(unique_results, key=lambda r: r["time"])
fastest_sp = single_ref["time"] / fastest_archi["time"] if fastest_archi["time"] > 0 else 0

print(f"\n🏆 Лучший MSE:    {best_archi['algo']}×{best_archi['n_islands']} "
      f"({best_archi['topology']}): MSE={best_archi['mse']:.6f}, speedup={best_sp:.2f}x")
print(f"🏆 Быстрейший:   {fastest_archi['algo']}×{fastest_archi['n_islands']} "
      f"({fastest_archi['topology']}): {fastest_archi['time']:.1f}s, speedup={fastest_sp:.2f}x")

print(f"\n✅ Этап 4 завершён. Переход к Этапу 5 (тюнинг solver).")

Config                       n_isl   Время      MSE            Speedup    Eff %   
----------------------------------------------------------------------------------
single (Stage 3)             1       17.17      0.001539       1.00x      —       
de1220×2 (ring)              2       33.05      0.002966       0.52x      26.0   %
de1220×4 (fully_connected)   4       27.06      0.001574       0.63x      15.9   %
de1220×4 (ring)              4       30.64      0.001635       0.56x      14.0   %
de1220×4 (unconnected)       4       22.76      0.001539       0.75x      18.9   %
de1220×8 (ring)              8       35.52      0.002055       0.48x      6.0    %
de1220×16 (ring)             16      39.03      0.001662       0.44x      2.7    %

🏆 Лучший MSE:    de1220×4 (unconnected): MSE=0.001539, speedup=0.75x
🏆 Быстрейший:   de1220×4 (unconnected): 22.8s, speedup=0.75x

✅ Этап 4 завершён. Переход к Этапу 5 (тюнинг solver).


### 4.5 Анализ результатов

**Scaling efficiency:**

| Фактор | Влияние |
|--------|---------|
| **n_islands ≤ cpu_count** | Линейный speedup — каждый остров на своём ядре, обход GIL через `mp_island` |
| **n_islands > cpu_count** | Diminishing returns — конкуренция за CPU, overhead переключения процессов |
| **Migration (ring/fully_connected)** | Улучшает MSE за счёт обмена лучшими решениями; `ring` — баланс diversification/exploitation |
| **Unconnected** | Чистый параллелизм без обмена; хорошо для speedup, но MSE может быть хуже |
| **Process overhead** | mp_island добавляет ~0.5-2с на создание процессов (заметно только на малых бюджетах) |

**Ключевые выводы:**
- `mp_island` даёт **настоящий multiprocessing** — кратный speedup по wall-clock time
- Оптимальное число островов ≈ `cpu_count` (больше — overhead без выгоды)
- `ring` топология — лучший баланс между качеством (MSE) и скоростью
- Для продакшена: archipelago + ring + cpu_count islands — рекомендуемая конфигурация

✅ Этап 4 завершён. Переход к Этапу 5 (тюнинг solver — влияние solver\_method на PyGMO).

## Этап 5: Тюнинг solver — влияние solver_method на PyGMO

Проверяем влияние **solver\_method** и **tolerance** в ODE на производительность PyGMO archipelago.

**Факторы:**
- **Solver:** BDF (stiff, точный), LSODA (автовыбор stiff/non-stiff), Radau (stiff, implicit Runge-Kutta)
- **Tolerance:** rtol=1e-2 (грубый, быстрый), rtol=1e-3 (средний), rtol=1e-4 (точный, медленный)

**Методика:** для каждой комбинации solver × tolerance создаём отдельный `ModelBasedUDP`,
запускаем archipelago с лучшей конфигурацией из Этапа 4, замеряем time и MSE.

**Цель:** найти оптимальную комбинацию solver + tolerance для продакшена.

### 5.1 Benchmark: solver × tolerance с archipelago

Для каждой комбинации (solver, rtol) создаём новый `ModelBasedUDP` и запускаем
archipelago с лучшей конфигурацией из Этапа 4:
- **Алгоритм:** `best_algo_name` (из Этапа 3)
- **Острова:** `best_n_islands` с топологией `ring`
- **Бюджет:** pop\_size=60, gen=20, n\_evolutions=4

In [18]:

# === Этап 5: Тюнинг solver — влияние solver_method на PyGMO ===

# Конфигурации для тестирования
solver_methods = ["BDF", "LSODA", "Radau"]
rtol_values = [1e-2, 1e-3, 1e-4]
ATOL = 1e-4  # фиксированный atol для всех тестов

# Лучшая конфигурация archipelago из Этапа 4
print(f"Конфигурация archipelago из Этапа 4:")
print(f"  Алгоритм:  {best_algo_name}")
print(f"  Островов:  {best_n_islands}")
print(f"  Топология: ring")
print(f"  Бюджет:    pop_size={POP_SIZE_3}, gen={N_GEN_3}, n_evolutions=4")
print(f"\nТестирование {len(solver_methods)} solvers × {len(rtol_values)} rtol = "
      f"{len(solver_methods) * len(rtol_values)} конфигураций\n")

results_solver = []

for method in solver_methods:
    for rtol in rtol_values:
        label = f"{method} rtol={rtol:.0e}"
        print(f"▶ {label}...", end=" ", flush=True)

        # Создаём UDP с конкретной конфигурацией solver
        udp_cfg = ModelBasedUDP(
            betas=betas,
            exp_temperature=exp_temperature,
            all_exp_masses=all_exp_masses,
            src_indices=src_indices,
            tgt_indices=tgt_indices,
            num_species=num_species,
            num_reactions=num_reactions,
            bounds_list=bounds_list,
            solver_method=method,
            solver_rtol=rtol,
            solver_atol=ATOL,
        )

        # Запускаем archipelago (переиспользуем run_archipelago с кастомным udp)
        gen_per_round = max(1, N_GEN_3 // 4)
        algo_map = {
            "de":       lambda s: pg.de(gen=gen_per_round, seed=s),
            "de1220":   lambda s: pg.de1220(gen=gen_per_round, seed=s),
            "sade":     lambda s: pg.sade(gen=gen_per_round, seed=s),
            "cmaes":    lambda s: pg.cmaes(gen=gen_per_round, seed=s, force_bounds=True),
            "pso":      lambda s: pg.pso(gen=gen_per_round, seed=s),
            "bee_colony": lambda s: pg.bee_colony(gen=gen_per_round, seed=s),
        }

        try:
            algo = pg.algorithm(algo_map[best_algo_name](42))
            prob = pg.problem(udp_cfg)
            topo = pg.topology(pg.ring())

            archi = pg.archipelago(t=topo)
            for i in range(best_n_islands):
                archi.push_back(
                    pg.island(algo=algo, prob=prob, size=POP_SIZE_3,
                              udi=pg.mp_island(), seed=42 + i)
                )

            t0 = time.perf_counter()
            for _ in range(4):  # n_evolutions=4
                archi.evolve()
                archi.wait()
            elapsed = time.perf_counter() - t0

            # Лучший champion
            champions_f = archi.get_champions_f()
            fitnesses = [f[0] for f in champions_f]
            best_mse = min(fitnesses)

            results_solver.append({
                "method": method,
                "rtol": rtol,
                "time": elapsed,
                "mse": best_mse,
                "label": label,
            })
            print(f"MSE={best_mse:.6f}, time={elapsed:.1f}s")

        except Exception as e:
            print(f"ОШИБКА: {e}")
            results_solver.append({
                "method": method,
                "rtol": rtol,
                "time": float("nan"),
                "mse": float("inf"),
                "label": label,
            })

print(f"\n✅ Benchmark завершён: {len(results_solver)} конфигураций протестировано.")

Конфигурация archipelago из Этапа 4:
  Алгоритм:  de1220
  Островов:  4
  Топология: ring
  Бюджет:    pop_size=60, gen=20, n_evolutions=4

Тестирование 3 solvers × 3 rtol = 9 конфигураций

▶ BDF rtol=1e-02... MSE=0.001702, time=73.2s
▶ BDF rtol=1e-03... MSE=0.001160, time=89.9s
▶ BDF rtol=1e-04... MSE=0.001786, time=97.3s
▶ LSODA rtol=1e-02... MSE=0.003010, time=28.9s
▶ LSODA rtol=1e-03... MSE=0.001803, time=32.1s
▶ LSODA rtol=1e-04... MSE=0.001558, time=34.4s
▶ Radau rtol=1e-02... MSE=0.003642, time=114.8s
▶ Radau rtol=1e-03... MSE=0.002850, time=116.9s
▶ Radau rtol=1e-04... MSE=0.003114, time=140.0s

✅ Benchmark завершён: 9 конфигураций протестировано.


### 5.2 Сравнительная таблица: solver × tolerance

Ранжирование конфигураций по:
1. **MSE** — качество итоговой оптимизации
2. **Время** — wall-clock time на archipelago
3. **Efficiency (MSE × time)** — баланс качества и скорости

In [19]:
# === СРАВНИТЕЛЬНАЯ ТАБЛИЦА ЭТАПА 5 ===

# Baseline: LSODA rtol=1e-2 (текущий default из pygmo_udp.py)
baseline_cfg = next(
    (r for r in results_solver if r["method"] == "LSODA" and r["rtol"] == 1e-2),
    results_solver[0]
)

# Сортировка по MSE
sorted_by_mse = sorted(results_solver, key=lambda r: r["mse"])

print(f"{'='*82}")
print(f"{'#':<4} {'Solver':<8} {'rtol':<10} {'Время (сек)':<14} {'MSE':<14} {'Speedup':<10} {'MSE×time':<12}")
print(f"{'-'*82}")
for rank, r in enumerate(sorted_by_mse, 1):
    sp = baseline_cfg["time"] / r["time"] if r["time"] > 0 else 0
    eff = r["mse"] * r["time"] if np.isfinite(r["mse"]) else float("inf")
    marker = " ★" if rank <= 3 else ""
    print(f"{rank:<4} {r['method']:<8} {r['rtol']:<10.0e} {r['time']:<14.2f} "
          f"{r['mse']:<14.6f} {f'{sp:.2f}x':<10} {eff:<12.4f}{marker}")
print(f"{'='*82}")

# Лучшая по MSE
best_solver = sorted_by_mse[0]
print(f"\n🏆 Лучший по MSE:        {best_solver['method']} rtol={best_solver['rtol']:.0e} "
      f"(MSE={best_solver['mse']:.6f}, {best_solver['time']:.1f}s)")

# Лучшая по efficiency
sorted_by_eff = sorted(results_solver, key=lambda r: r["mse"] * r["time"] if np.isfinite(r["mse"]) else float("inf"))
best_eff = sorted_by_eff[0]
print(f"🏆 Лучший по efficiency: {best_eff['method']} rtol={best_eff['rtol']:.0e} "
      f"(MSE×t={best_eff['mse'] * best_eff['time']:.4f})")

# Быстрейший
fastest = min(results_solver, key=lambda r: r["time"])
print(f"🏆 Быстрейший:          {fastest['method']} rtol={fastest['rtol']:.0e} "
      f"({fastest['time']:.1f}s, MSE={fastest['mse']:.6f})")

# Группировка по solver
print(f"\n--- Среднее по solver ---")
for method in solver_methods:
    method_results = [r for r in results_solver if r["method"] == method]
    avg_time = np.mean([r["time"] for r in method_results])
    avg_mse = np.mean([r["mse"] for r in method_results if np.isfinite(r["mse"])])
    print(f"  {method:<8}: avg_time={avg_time:.1f}s, avg_MSE={avg_mse:.6f}")

# Группировка по rtol
print(f"\n--- Среднее по rtol ---")
for rtol in rtol_values:
    rtol_results = [r for r in results_solver if r["rtol"] == rtol]
    avg_time = np.mean([r["time"] for r in rtol_results])
    avg_mse = np.mean([r["mse"] for r in rtol_results if np.isfinite(r["mse"])])
    print(f"  rtol={rtol:.0e}: avg_time={avg_time:.1f}s, avg_MSE={avg_mse:.6f}")

# Сохраняем лучшую конфигурацию для Этапа 6
best_solver_method = best_eff["method"]
best_solver_rtol = best_eff["rtol"]
print(f"\n📌 Лучшая конфигурация для Этапа 6: {best_solver_method} rtol={best_solver_rtol:.0e}")

#    Solver   rtol       Время (сек)    MSE            Speedup    MSE×time    
----------------------------------------------------------------------------------
1    BDF      1e-03      89.92          0.001160       0.32x      0.1043       ★
2    LSODA    1e-04      34.37          0.001558       0.84x      0.0536       ★
3    BDF      1e-02      73.20          0.001702       0.39x      0.1246       ★
4    BDF      1e-04      97.30          0.001786       0.30x      0.1737      
5    LSODA    1e-03      32.08          0.001803       0.90x      0.0578      
6    Radau    1e-03      116.86         0.002850       0.25x      0.3331      
7    LSODA    1e-02      28.91          0.003010       1.00x      0.0870      
8    Radau    1e-04      139.95         0.003114       0.21x      0.4358      
9    Radau    1e-02      114.81         0.003642       0.25x      0.4181      

🏆 Лучший по MSE:        BDF rtol=1e-03 (MSE=0.001160, 89.9s)
🏆 Лучший по efficiency: LSODA rtol=1e-04 (MSE×t=0.0536)
🏆 Б

### 5.3 Анализ результатов

**Влияние solver на производительность PyGMO:**

| Solver | Характеристика | Ожидание |
|--------|---------------|----------|
| **LSODA** | Автовыбор stiff/non-stiff метода. Минимальный overhead. | Быстрейший для большинства конфигураций |
| **BDF** | Implicit multi-step для stiff ODE. Точный, но дорогой. | Лучший MSE при высоком rtol, медленнее |
| **Radau** | Implicit Runge-Kutta (stiff). Хорош для жёстких задач. | Между BDF и LSODA по скорости |

**Влияние tolerance (rtol):**

| rtol | Точность | Скорость | Применение |
|------|----------|----------|------------|
| **1e-2** | Грубая | Максимальная | Exploration фаза, скрининг параметров |
| **1e-3** | Средняя | Средняя | Баланс качество/скорость |
| **1e-4** | Высокая | Минимальная | Финальный polish, валидация результатов |

**Рекомендация для продакшена:**
- **Exploration:** LSODA rtol=1e-2 — максимальная скорость при адекватном качестве
- **Exploitation/polish:** лучший solver с rtol=1e-3 или 1e-4 для финальной доводки
- **Двухфазная стратегия:** грубый solver → точный solver (аналог coarse-to-fine)

✅ Этап 5 завершён. Переход к Этапу 6 (финальный benchmark и рекомендации).